# 🚗 Python for Data Analysis: From Basics to Interactive Dashboards

## Lisbon Road Accidents Analysis Workshop (1h 40min)

**Course Objectives:**
- Learn to read and clean data from CSV files
- Perform statistical analysis on real-world data
- Create visualizations: graphs, plots, and maps
- Build an interactive dashboard with Streamlit

**Dataset:** Lisbon Road Accidents (2023)

---

## Part 1: Setup and Environment (10 min)

### 1.1 Install and Import Libraries

In [ ]:
# Install required packages
!pip install pandas numpy matplotlib seaborn geopandas folium plotly openpyxl -q

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import geopandas as gpd
import folium
from shapely.geometry import Point

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('viridis')
pd.set_option('display.max_columns', None)

print("✅ All libraries imported successfully!")

### 1.2 Upload the Dataset

In [ ]:
# Upload file to Colab
from google.colab import files

print("📁 Please upload the Road_Accidents_Lisbon.csv file:")
uploaded = files.upload()

---

## Part 2: Loading and Exploring Data (15 min)

### 2.1 Load and Preview Data

In [ ]:
# Load the CSV file
df = pd.read_csv('Road_Accidents_Lisbon.csv')

# Display basic information
print(f"📊 Dataset loaded: {len(df)} rows, {len(df.columns)} columns")
df.head()

In [ ]:
# Get dataset information
print("📋 Dataset Information:")
df.info()

In [ ]:
# Quick statistical summary
print("📊 Statistical Summary:")
df.describe()

In [ ]:
# List all columns
print("📑 Columns:")
for i, col in enumerate(df.columns, 1):
    print(f"   {i}. {col} ({df[col].dtype})")

### 💡 Exercise 1: Understanding the Data

Answer these questions:
1. How many accidents are in the dataset?
2. Which columns contain geographic information?
3. What types of data do we have (numerical vs categorical)?

---

## Part 3: Data Cleaning and Preparation (15 min)

### 3.1 Check Data Quality

In [ ]:
# Check for missing values
print("🔎 Missing Values:")
missing = df.isnull().sum()
print(missing[missing > 0] if missing.sum() > 0 else "   No missing values! ✅")

# Check for duplicates
print(f"\n🔄 Duplicate rows: {df.duplicated().sum()}")

### 3.2 Convert and Create Features

In [ ]:
# Convert date column
df['date'] = pd.to_datetime(df['date'])

# Extract time features
df['month'] = df['date'].dt.month
df['month_name'] = df['date'].dt.month_name()

print("📅 Date features created!")
df[['date', 'month', 'month_name']].head()

In [ ]:
# Create severity categories
def categorize_severity(row):
    if row['fatalities_30d'] > 0:
        return 'Fatal'
    elif row['injuries_serious'] > 0:
        return 'Serious'
    elif row['injuries_light'] > 0:
        return 'Light'
    else:
        return 'Property Damage Only'

df['severity'] = df.apply(categorize_severity, axis=1)

print("🏥 Severity distribution:")
print(df['severity'].value_counts())

In [ ]:
# Create time period categories
def categorize_time_period(hour):
    if 6 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 18:
        return 'Afternoon'
    elif 18 <= hour < 22:
        return 'Evening'
    else:
        return 'Night'

df['time_period'] = df['hour'].apply(categorize_time_period)

# Create total casualties
df['total_casualties'] = df['injuries_light'] + df['injuries_serious'] + df['fatalities_30d']

print("✅ All features created!")
print(f"   Total casualties in dataset: {df['total_casualties'].sum()}")

---

## Part 4: Statistical Analysis (15 min)

### 4.1 Frequency Analysis

In [ ]:
# Accidents by weather
print("🌤️ Accidents by Weather:")
print(df['weather'].value_counts())

In [ ]:
# Accidents by road type
print("🛣️ Accidents by Road Type:")
print(df['road_type'].value_counts())

In [ ]:
# Accidents by day of week
print("📅 Accidents by Day of Week:")
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
print(df['day_of_week'].value_counts().reindex(day_order))

### 4.2 Group-by Analysis

In [ ]:
# Top parishes by accidents
print("📍 Top 10 Parishes by Accidents:")
parish_stats = df.groupby('parish').agg({
    'id': 'count',
    'total_casualties': 'sum'
}).rename(columns={'id': 'accidents'}).sort_values('accidents', ascending=False)

parish_stats.head(10)

In [ ]:
# Hourly pattern
print("⏰ Accidents by Hour:")
hourly = df.groupby('hour')['id'].count()
print(f"   Peak hour: {hourly.idxmax()}:00 ({hourly.max()} accidents)")
print(f"   Lowest hour: {hourly.idxmin()}:00 ({hourly.min()} accidents)")

### 💡 Exercise 2: Statistical Questions

Find:
1. What is the most common accident type?
2. Which day of the week has the most accidents?
3. What is the average number of vehicles per accident?

In [ ]:
# Your answers:
print("1. Most common accident type:", df['accident_type'].mode()[0])
print("2. Day with most accidents:", df['day_of_week'].mode()[0])
print("3. Average vehicles per accident:", round(df['num_vehicles'].mean(), 2))

---

## Part 5: Data Visualization (20 min)

### 5.1 Bar Charts

In [ ]:
# Bar chart: Accidents by weather
fig, ax = plt.subplots(figsize=(10, 5))
weather_counts = df['weather'].value_counts()
bars = ax.bar(weather_counts.index, weather_counts.values, 
              color=sns.color_palette('viridis', len(weather_counts)))
ax.set_title('Accidents by Weather Condition', fontsize=14, fontweight='bold')
ax.set_xlabel('Weather')
ax.set_ylabel('Number of Accidents')

# Add value labels
for bar in bars:
    ax.annotate(f'{int(bar.get_height())}',
                xy=(bar.get_x() + bar.get_width()/2, bar.get_height()),
                ha='center', va='bottom')
plt.tight_layout()
plt.show()

In [ ]:
# Horizontal bar: Top parishes
fig, ax = plt.subplots(figsize=(10, 6))
top_parishes = df['parish'].value_counts().head(10)
ax.barh(top_parishes.index, top_parishes.values, color=sns.color_palette('viridis', 10))
ax.set_title('Top 10 Parishes by Accidents', fontsize=14, fontweight='bold')
ax.set_xlabel('Number of Accidents')
ax.invert_yaxis()
plt.tight_layout()
plt.show()

### 5.2 Pie Chart

In [ ]:
# Pie chart: Severity distribution
fig, ax = plt.subplots(figsize=(8, 8))
severity_counts = df['severity'].value_counts()
colors = ['#2ecc71', '#f1c40f', '#e74c3c', '#9b59b6']
ax.pie(severity_counts.values, labels=severity_counts.index,
       autopct='%1.1f%%', colors=colors, startangle=90,
       explode=(0.02, 0.02, 0.05, 0.1))
ax.set_title('Accident Severity Distribution', fontsize=14, fontweight='bold')
plt.show()

### 5.3 Line Chart

In [ ]:
# Line chart: Hourly distribution
fig, ax = plt.subplots(figsize=(12, 5))
hourly = df.groupby('hour')['id'].count()
ax.plot(hourly.index, hourly.values, marker='o', linewidth=2, color='#e74c3c')
ax.fill_between(hourly.index, hourly.values, alpha=0.3, color='#e74c3c')
ax.set_title('Accidents by Hour of Day', fontsize=14, fontweight='bold')
ax.set_xlabel('Hour')
ax.set_ylabel('Number of Accidents')
ax.set_xticks(range(0, 24))
ax.axvline(x=hourly.idxmax(), color='orange', linestyle='--', label=f'Peak: {hourly.idxmax()}:00')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### 5.4 Interactive Chart with Plotly

In [ ]:
# Interactive bar chart
fig = px.bar(df['accident_type'].value_counts().reset_index(),
             x='accident_type', y='count',
             title='Accidents by Type (Interactive)',
             labels={'accident_type': 'Accident Type', 'count': 'Count'},
             color='count', color_continuous_scale='Viridis')
fig.update_layout(xaxis_tickangle=-45)
fig.show()

---

## Part 6: Geospatial Visualization with Maps (15 min)

### 6.1 Create GeoDataFrame

In [ ]:
# Convert to GeoDataFrame
geometry = [Point(xy) for xy in zip(df['longitude'], df['latitude'])]
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs='EPSG:4326')

print(f"✅ GeoDataFrame created: {len(gdf)} points")
print(f"   Center: {gdf['latitude'].mean():.4f}, {gdf['longitude'].mean():.4f}")

### 6.2 Create Interactive Map

In [ ]:
# Create map centered on Lisbon
center = [gdf['latitude'].mean(), gdf['longitude'].mean()]
m = folium.Map(location=center, zoom_start=12, tiles='CartoDB Positron')

# Color mapping
severity_colors = {
    'Property Damage Only': 'green',
    'Light': 'orange',
    'Serious': 'red',
    'Fatal': 'darkred'
}

# Add markers
for _, row in gdf.iterrows():
    color = severity_colors.get(row['severity'], 'blue')
    popup = f"""
    <b>ID:</b> {row['id']}<br>
    <b>Date:</b> {row['date'].strftime('%Y-%m-%d')}<br>
    <b>Hour:</b> {row['hour']}:00<br>
    <b>Parish:</b> {row['parish']}<br>
    <b>Type:</b> {row['accident_type']}<br>
    <b>Severity:</b> {row['severity']}
    """
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=5,
        color=color,
        fill=True,
        fill_opacity=0.7,
        popup=folium.Popup(popup, max_width=250)
    ).add_to(m)

# Add legend
legend_html = '''
<div style="position: fixed; bottom: 50px; left: 50px; z-index: 1000; 
            background: white; padding: 10px; border: 2px solid grey; border-radius: 5px;">
    <b>Severity</b><br>
    <span style="color: green;">●</span> Property Damage<br>
    <span style="color: orange;">●</span> Light Injuries<br>
    <span style="color: red;">●</span> Serious<br>
    <span style="color: darkred;">●</span> Fatal
</div>
'''
m.get_root().html.add_child(folium.Element(legend_html))

m

### 💡 Exercise 3: Filter the Map

Create a map showing only serious and fatal accidents with larger markers.

In [ ]:
# Filter for serious/fatal accidents
severe_gdf = gdf[gdf['severity'].isin(['Serious', 'Fatal'])]
print(f"Filtering: {len(severe_gdf)} serious/fatal accidents")

# Create filtered map
m_severe = folium.Map(location=center, zoom_start=12, tiles='CartoDB dark_matter')

for _, row in severe_gdf.iterrows():
    color = 'darkred' if row['severity'] == 'Fatal' else 'red'
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=10,
        color=color,
        fill=True,
        fill_opacity=0.8,
        popup=f"{row['severity']} - {row['date'].strftime('%Y-%m-%d')}"
    ).add_to(m_severe)

m_severe

---

## Part 7: Exporting & Streamlit Preview (10 min)

### 7.1 Export Data

In [ ]:
# Export to CSV
df.to_csv('Road_Accidents_Processed.csv', index=False)
print("✅ CSV exported!")

# Export to Excel with multiple sheets
with pd.ExcelWriter('Road_Accidents_Analysis.xlsx', engine='openpyxl') as writer:
    df.to_excel(writer, sheet_name='All_Data', index=False)
    parish_stats.to_excel(writer, sheet_name='Parish_Stats')
print("✅ Excel exported!")

# Save map
m.save('accidents_map.html')
print("✅ Map saved!")

In [ ]:
# Download files
from google.colab import files
files.download('Road_Accidents_Processed.csv')
files.download('Road_Accidents_Analysis.xlsx')
files.download('accidents_map.html')

### 7.2 Streamlit Dashboard Preview

Now let's see how to combine everything into an interactive dashboard!

**Streamlit** is a Python framework that makes it easy to create web applications.

Below is the complete dashboard code. To run it:
1. Install Streamlit locally: `pip install streamlit streamlit-folium`
2. Save the code as `app.py`
3. Run: `streamlit run app.py`

In [ ]:
# Streamlit Dashboard Code - Save as app.py
streamlit_code = '''
import streamlit as st
import pandas as pd
import geopandas as gpd
import folium
import plotly.express as px
from shapely.geometry import Point
from streamlit_folium import st_folium

# Page config
st.set_page_config(page_title="Lisbon Accidents", page_icon="🚗", layout="wide")

# Load data
@st.cache_data
def load_data():
    df = pd.read_csv("data/Road_Accidents_Lisbon.csv")
    df[\'date\'] = pd.to_datetime(df[\'date\'])
    df[\'severity\'] = df.apply(lambda r: \'Fatal\' if r[\'fatalities_30d\'] > 0 
                                else (\'Serious\' if r[\'injuries_serious\'] > 0 
                                else (\'Light\' if r[\'injuries_light\'] > 0 else \'Property Damage\')), axis=1)
    df[\'total_casualties\'] = df[\'injuries_light\'] + df[\'injuries_serious\'] + df[\'fatalities_30d\']
    return df

df = load_data()

# Sidebar filters
st.sidebar.title("🔍 Filters")
severity_filter = st.sidebar.multiselect("Severity", df[\'severity\'].unique(), df[\'severity\'].unique())
weather_filter = st.sidebar.multiselect("Weather", df[\'weather\'].unique(), df[\'weather\'].unique())

# Apply filters
filtered = df[(df[\'severity\'].isin(severity_filter)) & (df[\'weather\'].isin(weather_filter))]

# Main content
st.title("🚗 Lisbon Road Accidents Dashboard")

# Metrics
c1, c2, c3, c4 = st.columns(4)
c1.metric("Accidents", len(filtered))
c2.metric("Casualties", filtered[\'total_casualties\'].sum())
c3.metric("Fatal", len(filtered[filtered[\'severity\'] == \'Fatal\']))
c4.metric("Parishes", filtered[\'parish\'].nunique())

# Map
st.subheader("📍 Accident Locations")
gdf = gpd.GeoDataFrame(filtered, geometry=[Point(xy) for xy in zip(filtered[\'longitude\'], filtered[\'latitude\'])], crs="EPSG:4326")
m = folium.Map(location=[gdf[\'latitude\'].mean(), gdf[\'longitude\'].mean()], zoom_start=12, tiles="CartoDB Positron")
colors = {\'Property Damage\': \'green\', \'Light\': \'orange\', \'Serious\': \'red\', \'Fatal\': \'darkred\'}
for _, r in gdf.iterrows():
    folium.CircleMarker([r[\'latitude\'], r[\'longitude\']], radius=5, color=colors.get(r[\'severity\'], \'blue\'),
                        fill=True, fill_opacity=0.7, popup=f"ID: {r[\'id\']} | {r[\'severity\']}").add_to(m)
st_folium(m, width=None, height=450)

# Charts
col1, col2 = st.columns(2)
with col1:
    fig = px.pie(filtered, names=\'severity\', title="Severity Distribution")
    st.plotly_chart(fig, use_container_width=True)
with col2:
    hourly = filtered.groupby(\'hour\').size().reset_index(name=\'count\')
    fig = px.line(hourly, x=\'hour\', y=\'count\', title="Hourly Distribution", markers=True)
    st.plotly_chart(fig, use_container_width=True)

# Download
st.download_button("📥 Download Data", filtered.to_csv(index=False), "accidents.csv", "text/csv")

st.markdown("---")
st.caption("Data for educational purposes only.")
'''

# Save the code
with open('app.py', 'w') as f:
    f.write(streamlit_code.replace("\\'", "'"))
print("✅ Streamlit app saved to app.py")

# Download
files.download('app.py')

---

## 🎯 Summary

### What We Learned:

| Part | Topic | Key Functions |
|------|-------|---------------|
| 1 | Setup | `import`, `pip install` |
| 2 | Loading Data | `pd.read_csv()`, `head()`, `info()` |
| 3 | Cleaning | `isnull()`, `apply()`, datetime conversion |
| 4 | Statistics | `value_counts()`, `groupby()`, `agg()` |
| 5 | Visualization | Matplotlib, Seaborn, Plotly |
| 6 | Maps | GeoPandas, Folium |
| 7 | Export & Dashboard | `to_csv()`, Streamlit |

### Next Steps:
1. Follow the PDF tutorial to set up Python locally
2. Run the Streamlit dashboard: `streamlit run app.py`
3. Customize and extend the dashboard!

---

**Thank you for attending! 🎉**